В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [13]:
# Імпортуємо бібліотеки
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Зчитуємо CSV-файл
df = pd.read_csv('/content/train.csv')

# Функція для препроцесингу
def preprocess_data(df, target_column="Exited", val_size=0.2, random_state=42):
    df = df.copy()

    # Видаляємо зайві колонки, якщо вони є
    drop_cols = ['RowNumber', 'CustomerId', 'Surname']
    drop_cols = [col for col in drop_cols if col in df.columns]
    df.drop(columns=drop_cols, inplace=True)

    # Розділяємо features та ціль
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Числові та категоріальні ознаки
    num_features = X.select_dtypes(include='number').columns.tolist()
    cat_features = X.select_dtypes(include='object').columns.tolist()

    # Побудова трансформера
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_features),
            ('cat', categorical_transformer, cat_features)
        ],
        remainder='passthrough' # Додано remainder='passthrough' для збереження інших колонок, якщо є
    )

    # Розділення на train і validation
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, random_state=random_state, stratify=y)

    # Навчання трансформера на тренувальних даних
    preprocessor.fit(X_train)

    # Трансформація
    X_train_processed = preprocessor.transform(X_train)
    X_val_processed = preprocessor.transform(X_val)

    # Повертаємо також препроцесор, числові та категоріальні ознаки
    return X_train_processed, y_train, X_val_processed, y_val, preprocessor, num_features, cat_features

# Препроцесинг
X_train, y_train, X_val, y_val, preprocessor, num_features, cat_features = preprocess_data(df)

# Модель KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_val)
print("🔹 KNN Accuracy:", accuracy_score(y_val, y_pred))

# Підбір гіперпараметрів
param_grid = {
    'n_neighbors': list(range(1, 21)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_
val_preds = best_model.predict(X_val)
print("✅ Best KNN params:", grid.best_params_)
print("✅ Best KNN validation accuracy:", accuracy_score(y_val, val_preds))

# Ансамбль
ensemble = VotingClassifier(
    estimators=[
        ('knn', best_model),
        ('tree', DecisionTreeClassifier(max_depth=5, random_state=42)),
        ('logreg', LogisticRegression(max_iter=1000, random_state=42))
    ],
    voting='soft'
)
ensemble.fit(X_train, y_train)
ensemble_preds = ensemble.predict(X_val)
print("🌟 Ensemble accuracy:", accuracy_score(y_val, ensemble_preds))

🔹 KNN Accuracy: 0.872
✅ Best KNN params: {'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'}
✅ Best KNN validation accuracy: 0.8776666666666667
🌟 Ensemble accuracy: 0.887


1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

🔬 Завдання: AUROC для kNN (параметри за замовченням)
✅ Пояснення:
AUROC — це площа під кривою ROC.

Вона вимірює здатність моделі відрізняти позитивний клас (Exited = 1) від негативного (Exited = 0).

Значення:

0.5 — випадкове вгадування

1.0 — ідеальна модель

0.8+ — вже добре

In [3]:
from sklearn.metrics import roc_auc_score

# kNN з параметрами за замовченням
knn_default = KNeighborsClassifier()
knn_default.fit(X_train, y_train)

# Прогнози ймовірностей
train_probs = knn_default.predict_proba(X_train)[:, 1]
val_probs = knn_default.predict_proba(X_val)[:, 1]

# Обчислення AUROC
train_auc = roc_auc_score(y_train, train_probs)
val_auc = roc_auc_score(y_val, val_probs)

print(f"🔵 AUROC на тренувальному наборі: {train_auc:.4f}")
print(f"🟠 AUROC на валідаційному наборі: {val_auc:.4f}")

🔵 AUROC на тренувальному наборі: 0.9598
🟠 AUROC на валідаційному наборі: 0.8677


🧪 Код для обчислення AUROC:

In [4]:
from sklearn.metrics import roc_auc_score

# kNN з параметрами за замовченням
knn_default = KNeighborsClassifier()
knn_default.fit(X_train, y_train)

# Прогнози ймовірностей
train_probs = knn_default.predict_proba(X_train)[:, 1]
val_probs = knn_default.predict_proba(X_val)[:, 1]

# Обчислення AUROC
train_auc = roc_auc_score(y_train, train_probs)
val_auc = roc_auc_score(y_val, val_probs)

print(f"🔵 AUROC на тренувальному наборі: {train_auc:.4f}")
print(f"🟠 AUROC на валідаційному наборі: {val_auc:.4f}")

🔵 AUROC на тренувальному наборі: 0.9598
🟠 AUROC на валідаційному наборі: 0.8677


📊 Як інтерпретувати результат
Після виконання коду, дивись:

Показник	Значення	Інтерпретація
AUROC (train)	високе (>0.9)?	Модель може бути overfitted (high variance)
AUROC (val)	низьке (<0.7)?	Модель underfits (high bias)
Обидва приблизно рівні, 0.8–0.9	👍	Хороший баланс

🧠 Висновок (залежно від результату):
Після запуску коду, ти можеш використати один з таких шаблонів:

🔹 Якщо train ≈ val ≈ 0.87–0.90:
Модель показує хороші результати як на тренувальному, так і на валідаційному наборі. Це означає, що вона не має значного перенавчання чи недонавчання. Можна сказати, що модель збалансована, має прийнятну здатність до узагальнення.

🔸 Якщо train дуже високий (0.95+), а val набагато нижчий (0.75-):
Модель демонструє високу точність на тренуванні, але гірші результати на валідації. Це вказує на перенавчання (high variance). Потрібно зменшити складність моделі або використати регуляризацію/ансамблювання.

🔻 Якщо обидва AUROC < 0.7:
Модель має низьку здатність до відокремлення класів. Це вказує на high bias — вона недостатньо потужна, щоб вловити закономірності. Можна спробувати інші моделі або інженерію ознак.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

# Визначення сітки параметрів для GridSearchCV
param_grid_knn = {'n_neighbors': list(range(1, 21))}

# Ініціалізація GridSearchCV
grid_search_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5, scoring='roc_auc', n_jobs=-1)

# Проведення пошуку по сітці
grid_search_knn.fit(X_train, y_train)

# Виведення найкращого значення параметра
print("✅ Найкраще значення n_neighbors:", grid_search_knn.best_params_['n_neighbors'])

# Збереження найкращої моделі
knn_best = grid_search_knn.best_estimator_

# Оцінка якості передбачень knn_best на тренувальній і валідаційній вибірках за допомогою AUROC
train_probs_best_knn = knn_best.predict_proba(X_train)[:, 1]
val_probs_best_knn = knn_best.predict_proba(X_val)[:, 1]

train_auc_best_knn = roc_auc_score(y_train, train_probs_best_knn)
val_auc_best_knn = roc_auc_score(y_val, val_probs_best_knn)

print(f"🔵 AUROC на тренувальному наборі (knn_best): {train_auc_best_knn:.4f}")
print(f"🟠 AUROC на валідаційному наборі (knn_best): {val_auc_best_knn:.4f}")

# Висновок про якість моделі
print("\n📊 Висновок:")
if val_auc_best_knn > val_auc: # Comparing with the previous default KNN AUROC
    print("Модель kNN з оптимальним n_neighbors показала кращий результат на валідаційному наборі порівняно з моделлю з параметрами за замовчуванням.")
elif val_auc_best_knn == val_auc:
     print("Модель kNN з оптимальним n_neighbors показала такий же результат на валідаційному наборі порівняно з моделлю з параметрами за замовчуванням.")
else:
    print("Модель kNN з оптимальним n_neighbors показала гірший результат на валідаційному наборі порівняно з моделлю з параметрами за замовчуванням.")

# Note: To compare with the Decision Tree from the previous task, we would need its AUROC score from that task.
# Assuming we don't have that readily available in this context, we can only compare the current KNN models.

✅ Найкраще значення n_neighbors: 20
🔵 AUROC на тренувальному наборі (knn_best): 0.9358
🟠 AUROC на валідаційному наборі (knn_best): 0.9102

📊 Висновок:
Модель kNN з оптимальним n_neighbors показала кращий результат на валідаційному наборі порівняно з моделлю з параметрами за замовчуванням.


3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [6]:
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

# Визначення сітки параметрів для GridSearchCV
param_grid_dt = {
    'max_depth': list(range(1, 21, 2)),
    'max_leaf_nodes': list(range(2, 11, 1))
}

# Ініціалізація DecisionTreeClassifier з фіксацією random_state
dt = DecisionTreeClassifier(random_state=42)

# Ініціалізація GridSearchCV
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=3, scoring='roc_auc', n_jobs=-1)

# Вимірювання часу пошуку
start_time = time.time()
grid_search_dt.fit(X_train, y_train)
end_time = time.time()

# Виведення найкращих значень параметрів
print("✅ Найкращі значення гіперпараметрів для DecisionTreeClassifier (GridSearchCV):", grid_search_dt.best_params_)

# Збереження найкращої моделі
dt_best = grid_search_dt.best_estimator_

# Оцінка якості передбачень dt_best на тренувальній і валідаційній вибірках за допомогою AUROC
train_probs_best_dt = dt_best.predict_proba(X_train)[:, 1]
val_probs_best_dt = dt_best.predict_proba(X_val)[:, 1]

train_auc_best_dt = roc_auc_score(y_train, train_probs_best_dt)
val_auc_best_dt = roc_auc_score(y_val, val_probs_best_dt)

print(f"🔵 AUROC на тренувальному наборі (dt_best): {train_auc_best_dt:.4f}")
print(f"🟠 AUROC на валідаційному наборі (dt_best): {val_auc_best_dt:.4f}")

print(f"⏱ Час, витрачений на пошук гіперпараметрів: {end_time - start_time:.2f} секунд")

# Висновок про якість моделі
print("\n📊 Висновок:")
# For comparison with a manually found Decision Tree, we would need its AUROC score.
# Assuming we don't have that explicitly stated in the current context, we compare with the previous best KNN.
# To compare with a manual DT, you would need to add that manual DT's AUROC here.
print(f"Модель Decision Tree з оптимальними гіперпараметрами (GridSearchCV) має AUROC на валідаційному наборі: {val_auc_best_dt:.4f}.")
# You can add a comparison with the best KNN here if desired, e.g.:
# if val_auc_best_dt > val_auc_best_knn:
#    print(f"Ця модель Decision Tree краща за найкращу модель kNN (AUROC на валідації {val_auc_best_knn:.4f}).")
# elif val_auc_best_dt < val_auc_best_knn:
#    print(f"Ця модель Decision Tree гірша за найкращу модель kNN (AUROC на валідації {val_auc_best_knn:.4f}).")
# else:
#    print(f"Ця модель Decision Tree має схожу якість з найкращою моделлю kNN (AUROC на валідації {val_auc_best_knn:.4f}).")

✅ Найкращі значення гіперпараметрів для DecisionTreeClassifier (GridSearchCV): {'max_depth': 5, 'max_leaf_nodes': 10}
🔵 AUROC на тренувальному наборі (dt_best): 0.9015
🟠 AUROC на валідаційному наборі (dt_best): 0.9002
⏱ Час, витрачений на пошук гіперпараметрів: 4.55 секунд

📊 Висновок:
Модель Decision Tree з оптимальними гіперпараметрами (GridSearchCV) має AUROC на валідаційному наборі: 0.9002.


4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [7]:
import time
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

# Визначення сітки параметрів для RandomizedSearchCV
params_dt_random = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

# Ініціалізація DecisionTreeClassifier з фіксацією random_state
dt_random = DecisionTreeClassifier(random_state=42)

# Ініціалізація RandomizedSearchCV
random_search_dt = RandomizedSearchCV(dt_random, params_dt_random, n_iter=40, cv=3, scoring='roc_auc', random_state=42, n_jobs=-1)

# Вимірювання часу пошуку
start_time_random = time.time()
random_search_dt.fit(X_train, y_train)
end_time_random = time.time()

# Виведення найкращих значень параметрів
print("✅ Найкращі значення гіперпараметрів для DecisionTreeClassifier (RandomizedSearchCV):", random_search_dt.best_params_)

# Збереження найкращої моделі
dt_random_search_best = random_search_dt.best_estimator_

# Оцінка якості передбачень dt_random_search_best на тренувальній і валідаційній вибірках за допомогою AUROC
train_probs_best_dt_random = dt_random_search_best.predict_proba(X_train)[:, 1]
val_probs_best_dt_random = dt_random_search_best.predict_proba(X_val)[:, 1]

train_auc_best_dt_random = roc_auc_score(y_train, train_probs_best_dt_random)
val_auc_best_dt_random = roc_auc_score(y_val, val_probs_best_dt_random)

print(f"🔵 AUROC на тренувальному наборі (dt_random_search_best): {train_auc_best_dt_random:.4f}")
print(f"🟠 AUROC на валідаційному наборі (dt_random_search_best): {val_auc_best_dt_random:.4f}")

print(f"⏱ Час, витрачений на пошук гіперпараметрів (RandomizedSearchCV): {end_time_random - start_time_random:.2f} секунд")

# Висновок про якість моделі та порівняння з GridSearch
print("\n📊 Висновок:")
print(f"Модель Decision Tree з оптимальними гіперпараметрами (RandomizedSearchCV) має AUROC на валідаційному наборі: {val_auc_best_dt_random:.4f}.")

# Порівняння з GridSearch
if val_auc_best_dt_random > val_auc_best_dt:
    print(f"Ця модель Decision Tree, знайдена за допомогою RandomizedSearchCV, краща за модель, знайдену за допомогою GridSearchCV (AUROC на валідації {val_auc_best_dt:.4f}).")
elif val_auc_best_dt_random < val_auc_best_dt:
     print(f"Ця модель Decision Tree, знайдена за допомогою RandomizedSearchCV, гірша за модель, знайдену за допомогою GridSearchCV (AUROC на валідації {val_auc_best_dt:.4f}).")
else:
    print(f"Ця модель Decision Tree, знайдена за допомогою RandomizedSearchCV, має схожу якість з моделлю, знайденою за допомогою GridSearchCV (AUROC на валідації {val_auc_best_dt:.4f}).")

print("\n🔍 Аналіз параметрів:")
print("Параметри dt_random_search_best:", dt_random_search_best.get_params())
print("Параметри dt_best (з GridSearch):", dt_best.get_params())
print("\nВідмінності в параметрах можуть вказувати на те, як RandomizedSearchCV досліджував простір параметрів і які комбінації він вважав оптимальними в межах заданої кількості ітерацій порівняно з вичерпним пошуком GridSearchCV.")

✅ Найкращі значення гіперпараметрів для DecisionTreeClassifier (RandomizedSearchCV): {'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': np.int64(14), 'max_features': None, 'max_depth': np.int64(16), 'criterion': 'entropy'}
🔵 AUROC на тренувальному наборі (dt_random_search_best): 0.9169
🟠 AUROC на валідаційному наборі (dt_random_search_best): 0.9166
⏱ Час, витрачений на пошук гіперпараметрів (RandomizedSearchCV): 1.24 секунд

📊 Висновок:
Модель Decision Tree з оптимальними гіперпараметрами (RandomizedSearchCV) має AUROC на валідаційному наборі: 0.9166.
Ця модель Decision Tree, знайдена за допомогою RandomizedSearchCV, краща за модель, знайдену за допомогою GridSearchCV (AUROC на валідації 0.9002).

🔍 Аналіз параметрів:
Параметри dt_random_search_best: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': np.int64(16), 'max_features': None, 'max_leaf_nodes': np.int64(14), 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [15]:
# Завантажуємо тестові дані
# Припускаємо, що тестовий файл називається 'test.csv' і знаходиться в тому ж каталозі, що й train.csv
# Якщо назва файлу інша або він знаходиться в іншому місці, будь ласка, змініть шлях
try:
    df_test = pd.read_csv('/content/test.csv')
except FileNotFoundError:
    print("Помилка: Файл 'test.csv' не знайдено. Переконайтеся, що файл знаходиться за вказаним шляхом.")
    # Можна додати код для завантаження файлу, якщо він доступний за URL, або попросити користувача завантажити його.
    # Наприклад: !wget -O /content/test.csv [URL до test.csv]
    # Для цього прикладу, якщо файл не знайдено, зупинимо виконання
    raise

# Зберігаємо CustomerId для файлу submission
submission_ids = df_test['CustomerId']

# Видаляємо ті ж колонки, що видаляли з тренувальних даних
drop_cols_test = ['RowNumber', 'CustomerId', 'Surname']
drop_cols_test = [col for col in drop_cols_test if col in df_test.columns]
# Зберігаємо сирі тестові дані для препроцесингу
X_test_raw = df_test.drop(columns=drop_cols_test)


# Застосовуємо НАВЧЕНИЙ препроцесор до тестових даних
# Тепер об'єкт 'preprocessor' повинен бути доступний з першої комірки після її модифікації та виконання
preprocessor_available = False
try:
    # Перевіряємо, чи об'єкт preprocessor доступний
    _ = preprocessor
    X_test_processed = preprocessor.transform(X_test_raw)
    print("✅ Препроцесинг тестових даних виконано за допомогою навченого препроцесора.")
    preprocessor_available = True
except NameError:
    print("Помилка: Об'єкт 'preprocessor' недоступний. Спроба відтворити трансформер...")
    # Якщо препроцесор все ще недоступний, спробуємо відтворити його, використовуючи збережені num_features та cat_features
    try:
        _ = num_features
        _ = cat_features

        numeric_transformer_submission = StandardScaler()
        categorical_transformer_submission = OneHotEncoder(handle_unknown='ignore')

        preprocessor_submission_recreated = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer_submission, num_features),
                ('cat', categorical_transformer_submission, cat_features)
            ],
            remainder='passthrough'
        )

        # Завантажуємо train.csv знову, щоб отримати X_train без препроцесингу для навчання відтвореного препроцесора
        try:
            df_train_raw = pd.read_csv('/content/train.csv')
            drop_cols_train_raw = ['RowNumber', 'CustomerId', 'Surname', 'Exited']
            drop_cols_train_raw = [col for col in drop_cols_train_raw if col in df_train_raw.columns]
            X_train_raw = df_train_raw.drop(columns=drop_cols_train_raw)

            preprocessor_submission_recreated.fit(X_train_raw)

            X_test_processed = preprocessor_submission_recreated.transform(X_test_raw)
            print("✅ Препроцесинг тестових даних виконано за допомогою відтвореного препроцесора.")
            preprocessor_available = True

        except FileNotFoundError:
             print("Помилка: Файл 'train.csv' не знайдено під час спроби відтворення препроцесора.")
        except NameError:
             print("Помилка: Змінні num_features або cat_features недоступні для відтворення препроцесора.")

    except NameError:
        print("Критична помилка: Не вдалося отримати доступ до необхідних змінних для препроцесингу.")


# Отримуємо передбачення ймовірностей від найкращої моделі (dt_random_search_best)
# Використовуємо predict_proba для отримання ймовірності позитивного класу (Exited=1)
if preprocessor_available: # Перевіряємо, чи успішно виконано препроцесинг
    if 'dt_random_search_best' in locals():
        test_probabilities = dt_random_search_best.predict_proba(X_test_processed)[:, 1]
        print("✅ Передбачення ймовірностей для тестових даних отримано.")

        # Створюємо DataFrame для submission файлу
        submission_df = pd.DataFrame({'CustomerId': submission_ids, 'Exited': test_probabilities})

        # Зберігаємо файл submission.csv
        submission_df.to_csv('submission.csv', index=False)

        print("\n🎉 Файл submission.csv успішно створено!")
        print("Тепер ви можете завантажити файл 'submission.csv' на Kaggle для оцінки.")

        # Для додавання скріншоту скора на публічному лідерборді, вам потрібно буде зробити це вручну
        # після завантаження файлу та отримання результату на Kaggle.
    else:
        print("Помилка: Змінна 'dt_random_search_best' не знайдена. Переконайтеся, що ви успішно виконали попередні кроки.")
else:
    print("Помилка: Препроцесинг тестових даних не був успішним. Неможливо отримати передбачення.")

✅ Препроцесинг тестових даних виконано за допомогою навченого препроцесора.
✅ Передбачення ймовірностей для тестових даних отримано.

🎉 Файл submission.csv успішно створено!
Тепер ви можете завантажити файл 'submission.csv' на Kaggle для оцінки.
